# 0.0 IMPORTS

In [20]:
import pandas as pd
import inflection
import math

## 0.1. Helper functions

## 0.2 Loading dada

In [4]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

# merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

In [5]:
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
269349,300,3,2014-11-26,6279,827,1,1,0,0,a,c,120.0,4.0,2009.0,1,1.0,2012.0,"Jan,Apr,Jul,Oct"


# 1.0 DATA DESCRIPTION

In [7]:
# fazer uma copia para ficar mais fácil de trabalhar com os dados
# aí não dá problema de ter que refazer as coisas no dataset
df1 = df_raw.copy()

## 1.1. Rename columns
Renomear as colunas para ficar mais fácil de trabalhar com elas.

In [8]:
df1.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [13]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 
            'SchoolHoliday', 'StoreType', 'Assortment','CompetitionDistance', 'CompetitionOpenSinceMonth', 
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

# snake_case
snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

# rename
df1.columns = cols_new

In [14]:
# verificando as colunas renomeadas
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

## 1.2.  Data dimensions
Saber qual a quantidade de linhas e colunas do dataset.

In [15]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of cols: {}'.format(df1.shape[1]))

Number of rows: 1017209
Number of cols: 18


## 1.3.  Data types
Verificando os tipos de dados.

In [16]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [18]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4.  Check NaN
Verificando a existência de dados faltantes.

In [19]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

Observamos que temos vários dados faltantes. Podemos seguir por três caminhos:

1) **Excluir todas as linhas que possuem esses dados**, mas podemos perder informações importantes que o algoritmo de ML precisa e impactar negativamente nas previsões.

2) **Usar técnicas de ML** para preencher esses dados faltantes e prever qual valor seria no lugar dos `NaN`. Esse método é bom quando não temos informações suficientes de Negócios.

3) **Realmente entender o Negócio**. Entender as regras de negócio e preencher os dados faltantes com informações relevantes.

## 1.5.  Fillout NA
Vamos pensar no método de entender o negócio para fazer o preenchimento. Ver a descrição de cada coluna no Kaggle.

https://www.kaggle.com/c/rossmann-store-sales/data

Data fields

Most of the fields are self-explanatory. The following are descriptions for those that aren't.

- **Id** - an Id that represents a (Store, Date) duple within the test set
- **Store** - a unique Id for each store
- **Sales** - the turnover for any given day (this is what you are predicting)
- **Customers** - the number of customers on a given day
- **Open** - an indicator for whether the store was open: 0 = closed, 1 = open
- **StateHoliday** - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- **SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools
- **StoreType** - differentiates between 4 different store models: a, b, c, d
- **Assortment** - describes an assortment level: a = basic, b = extra, c = extended
- **CompetitionDistance** - distance in meters to the nearest competitor store
- **CompetitionOpenSince[Month/Year]** - gives the approximate year and month of the time the nearest competitor was opened
- **Promo** - indicates whether a store is running a promo on that day
- **Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- **Promo2Since[Year/Week]** - describes the year and calendar week when the store started participating in Promo2
- **PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store


In [21]:
# observando qual é a distância máxima até o competidor mais próximo
df1['competition_distance'].max()

75860.0

In [33]:
# competition_distance
# vamos assumir que a distância dos competidores e tão grande que não há competidores
# aplicando para cada registro da coluna: competition_distance
#     será 200000.0 se registro == NaN ; será o próprio valor se registro != NaN
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)

# competition_open_since_month
# vamos assumir que a data de abertura é a data da loja: date
# axis=1 -> aplicar na linha ao longo das colunas, pois temos mais de uma coluna na expressão
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

# competition_open_since_year 
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

# promo2_since_week
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

# promo2_since_year
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

In [ ]:
# promo_interval

# depois se a data estiver dentro da lista, cria uma coluna falando que a promo2 tava ativa

# criando dicionario que vai ajudar no mapeamento
month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
             7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

# preenchendo com 0 os valores NaN
df1['promo_interval'].fillna(0, inplace=True)

# criando a coluna 'month_map' como mapeamento do mês da coluna 'date'
df1['month_map'] = df1['date'].dt.month.map(month_map)

In [ ]:
df1.sample(5).T

In [40]:
# faz um split coluna 'promo_interval' e cria uma lista contendo os valores
# verifica se o valor de 'month_map' esta dentro dessa lista
# cria a coluna 'is_promo' com valores numéricos (1 = estava em promo ou 0 = não estava em promo) com base na verificação
df1['is_promo'] = df1[['promo_interval','month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [43]:
df1.sample(5).T

,96508,391905,979061,476637,994117
store,619,377,877,203,323
day_of_week,3,5,1,3,1
date,2015-05-06 00:00:00,2014-07-18 00:00:00,2013-02-04 00:00:00,2014-04-30 00:00:00,2013-01-21 00:00:00
sales,6131,8243,7739,9610,6980
customers,468,982,962,1236,579
open,1,1,1,1,1
promo,1,1,1,1,1
state_holiday,0,0,0,0,0
school_holiday,0,1,0,0,0
store_type,a,a,a,c,d


In [44]:
# verificando operações
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64

## 1.1. 

## 1.1. 

## 1.1. 